In [225]:
import pandas as pd

In [226]:
import numpy as np

In [227]:
import csv

In [228]:
from sklearn.model_selection import train_test_split
all_data = pd.read_csv('data.csv',index_col=0)
all_data.head()
print(all_data)

          diagnostico  radio_medio  textura_media  perimetro_medio  \
id                                                                   
842302              0        17.99          10.38           122.80   
842517              0        20.57          17.77           132.90   
84300903            0        19.69          21.25           130.00   
84348301            0        11.42          20.38            77.58   
84358402            0        20.29          14.34           135.10   
...               ...          ...            ...              ...   
926424              0        21.56          22.39           142.00   
926682              0        20.13          28.25           131.20   
926954              0        16.60          28.08           108.30   
927241              0        20.60          29.33           140.10   
92751               1         7.76          24.54            47.92   

          area_media  suavidad_media  compactacion_media  concavidad_media  \
id         

In [229]:
print(all_data.columns)

Index(['diagnostico', 'radio_medio', 'textura_media', 'perimetro_medio',
       'area_media', 'suavidad_media', 'compactacion_media',
       'concavidad_media', 'puntos_concavos_medios', 'simetria_media',
       'dimension_fractal_media', 'radio_se', 'textura_se', 'perimetro_se',
       'area_se', 'suavidad_se', 'compactacion_se', 'concavidad_se',
       'concavidad_puntos_se', 'simetria_se', 'fractal_dimension_se',
       'radio_peor', 'textura_peor', 'perimetro_peor', 'area_peor',
       'suavidad_peor', 'compactacion_peor', 'concavidad_peor',
       'concavidad_punto_peor', 'simetria_peor', 'fractal_dimension_peor'],
      dtype='object')


In [230]:
all_inputs = all_data.iloc[:, 1:32].values
all_outputs = all_data.iloc[:, 0].values

# Dividir en un conjunto de entrenamiento y uno de prueba
X_train, X_test, Y_train, Y_test = train_test_split(all_inputs, all_outputs,
    test_size=1/30)

n = X_train.shape[0] # número de registros de entrenamiento



In [231]:
print(all_outputs)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 0
 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0
 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 1 0 0 1 1
 1 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1
 1 1 0 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0
 0 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 0 1 1 1 1 1 0 1 1
 0 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 0 1 0 1 1 0 

In [232]:
print(all_inputs)


[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e+02 2.750e+02 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e+02 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+02 ... 2.650e+02 4.087e-01 1.240e+02]
 [7.760e+00 2.454e+01 4.792e+01 ... 0.000e+00 2.871e-01 7.039e-02]]


In [233]:
print(all_outputs)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 0
 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0
 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 1 0 0 1 1
 1 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1
 1 1 0 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0
 0 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 0 1 1 1 1 1 0 1 1
 0 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 0 1 0 1 1 0 

In [234]:
n = X_train.shape[0] # número de registros de entrenamiento

In [235]:
 #Construir una red neuronal con pesos y sesgos
#w_hidden = np.random.rand(3, 3)
w_hidden = np.random.rand(30, 30)
w_output = np.random.rand(1, 30)

b_hidden = np.random.rand(30, 1)
b_output = np.random.rand(1, 1)


In [236]:
X_train, X_test, Y_train, Y_test = train_test_split(all_inputs, all_outputs,
    test_size=1/3)

n = X_train.shape[0] # número de registros de entrenamiento


In [237]:
# Funciones de activacion
relu = lambda x: np.maximum(x, 0)
logistic = lambda x: 1 / (1 + np.exp(-x))

In [238]:
# Funcion que corre la red neuronal con los datos de entrada para predecir la salida
def forward_prop(X):
    Z1 = w_hidden @ X + b_hidden
    A1 = relu(Z1)
    Z2 = w_output @ A1 + b_output
    A2 = logistic(Z2)
    return Z1, A1, Z2, A2


In [239]:
test_predictions = forward_prop(X_test.transpose())[3] # me interesa solo la capa de salida, A2
test_comparisons = np.equal((test_predictions >= .5).flatten().astype(int), Y_test)
accuracy = sum(test_comparisons.astype(int) / X_test.shape[0])
print("ACCURACY: ", accuracy)


ACCURACY:  0.6157894736842114


In [240]:
w_hidden = np.random.rand(30, 30)
print('w_hidden = ', w_hidden)



w_hidden =  [[9.26378989e-01 5.86862641e-01 3.58129088e-01 8.80848848e-01
  1.63255813e-01 5.51889987e-01 7.27551824e-01 1.92277333e-01
  1.13916975e-01 3.25834404e-01 7.19698734e-01 7.06285575e-01
  2.68606487e-01 6.69102807e-01 3.68431073e-01 2.60749453e-01
  6.19992863e-01 7.60781164e-03 8.69952875e-01 4.64064322e-01
  9.45974607e-01 2.81668323e-01 4.13802850e-01 1.93895157e-01
  9.79702530e-03 9.86944495e-01 2.62531231e-01 5.70930577e-01
  5.39058895e-01 8.97329539e-01]
 [4.35372631e-01 1.81588871e-01 1.66465074e-01 4.38202880e-01
  7.01305940e-01 6.09991241e-01 4.64751602e-01 8.51205998e-01
  5.57897883e-01 9.38372554e-01 7.61930157e-01 4.50928234e-01
  4.67076296e-01 7.49716861e-01 4.95866557e-01 8.46810458e-01
  8.59336091e-01 1.61885484e-01 1.13290030e-01 7.74992945e-01
  2.71133221e-01 3.69815323e-01 8.19453186e-01 6.41147785e-01
  8.08233100e-01 4.56959253e-01 9.73979541e-01 3.46957500e-01
  7.86778440e-01 2.93170394e-01]
 [9.04241024e-01 9.05253498e-01 7.57059357e-01 4.16438

In [241]:
w_output = np.random.rand(1, 30)
print('w_output = ', w_output)

w_output =  [[9.74351896e-01 1.73050219e-02 1.39259337e-02 8.17384075e-01
  9.50811502e-03 3.82414890e-01 1.02749887e-01 5.31066211e-04
  1.33173313e-02 9.23503887e-01 2.18988918e-01 7.73071036e-03
  4.09726189e-01 2.50932230e-02 6.79364387e-01 7.80595551e-01
  3.30900541e-01 7.25941179e-02 4.26298731e-01 7.06981346e-01
  3.53315111e-01 5.37986689e-02 6.55193889e-01 3.42828496e-01
  7.37613901e-01 1.23182947e-01 7.44182056e-01 4.74159228e-01
  8.61194430e-01 2.05691803e-01]]


In [242]:
b_hidden = np.random.rand(30, 1)
print('b_hidden = ', b_hidden)



b_hidden =  [[0.40344872]
 [0.16530054]
 [0.65671068]
 [0.21086107]
 [0.97707227]
 [0.342251  ]
 [0.75193361]
 [0.36055177]
 [0.35763207]
 [0.61233784]
 [0.49679266]
 [0.78402406]
 [0.4817067 ]
 [0.16871937]
 [0.92454536]
 [0.29203404]
 [0.88727458]
 [0.92412153]
 [0.7375617 ]
 [0.75864695]
 [0.71780465]
 [0.80563838]
 [0.21762768]
 [0.64406038]
 [0.30464036]
 [0.44765107]
 [0.68761682]
 [0.41971056]
 [0.21150992]
 [0.95093683]]


In [243]:
b_output = np.random.rand(1, 1)
print('b_output = ', b_output)

b_output =  [[0.16560577]]


In [244]:

from sympy import *

W1, W2, B1, B2, A1, A2, Z1, Z2, X, Y = \
    symbols('W1 W2 B1 B2 A1 A2 Z1 Z2 X Y')

# Derivada de la función costo respecto A2
C = (A2 - Y)**2
dC_dA2 = diff(C, A2)
print("dC_dA2 = ", dC_dA2) # 2*A2 - 2*Y

# Derivada de A2 respecto de Z2
logistic = lambda x: 1 / (1 + exp(-x))
_A2 = logistic(Z2)
dA2_dZ2 = diff(_A2, Z2)
print("dA2_dZ2 = ", dA2_dZ2) # exp(-Z2)/(1 + exp(-Z2))**2

# Derivada de Z2 respecto a A1
_Z2 = A1*W2 + B2
dZ2_dA1 = diff(_Z2, A1)
print("dZ2_dA1 = ", dZ2_dA1) # W2

# Derivada de Z2 respecto a W2
dZ2_dW2 = diff(_Z2, W2)
print("dZ2_dW2 = ", dZ2_dW2) # A1

# Derivada de Z2 respecto a B2
dZ2_dB2 = diff(_Z2, B2)
print("dZ2_dB2 = ", dZ2_dB2) # 1

# Derivada de A1 respecto de Z1
relu = lambda x: Max(x, 0)
_A1 = relu(Z1)

d_relu = lambda x: x > 0 # Pendiente es 1 para los positivos, 0 para los negativos
dA1_dZ1 = d_relu(Z1)
print("dA1_dZ1 = ", dA1_dZ1) # Z1 > 0

# Derivada de Z1 respecto a W1
_Z1 = X*W1 + B1
dZ1_dW1 = diff(_Z1, W1)
print("dZ1_dW1 = ", dZ1_dW1) # X

# Derivada de Z1 respecto a B1
dZ1_dB1 = diff(_Z1, B1)
print("dZ1_dB1 = ", dZ1_dB1) # 1

dC_dA2 =  2*A2 - 2*Y
dA2_dZ2 =  exp(-Z2)/(1 + exp(-Z2))**2
dZ2_dA1 =  W2
dZ2_dW2 =  A1
dZ2_dB2 =  1
dA1_dZ1 =  Z1 > 0
dZ1_dW1 =  X
dZ1_dB1 =  1


In [245]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
all_data = pd.read_csv('data.csv',index_col=0)



L = 0.05  # La tasa de aprendizaje

all_inputs = all_data.iloc[:, 1:32].values
all_outputs = all_data.iloc[:, 0].values


# Dividir los conjuntos de datos de entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(all_inputs, all_outputs,
    test_size=1 / 3)
n = X_train.shape[0]
print(n)

# Construir red neuronal con pesos y sesgos
# inicializados aleatoriamente
w_hidden = np.random.rand(30, 30)
w_output = np.random.rand(1, 30)

b_hidden = np.random.rand(30, 1)
b_output = np.random.rand(1, 1)

# Funciones de activación
relu = lambda x: np.maximum(x, 0)
logistic = lambda x: 1 / (1 + np.exp(-x))

# Ejecutar entradas a través de la red neuronal para obtener salidas predichas
def forward_prop(X):
    Z1 = w_hidden @ X + b_hidden
    A1 = relu(Z1)
    Z2 = w_output @ A1 + b_output
    A2 = logistic(Z2)
    return Z1, A1, Z2, A2

# Derivadas de las funciones de activación
d_relu = lambda x: x > 0
d_logistic = lambda x: np.exp(-x) / (1 + np.exp(-x)) ** 2

# Devuelve pendientes para pesos y sesgos
# usando la regla de la cadena
def backward_prop(Z1, A1, Z2, A2, X, Y):
    dC_dA2 = 2 * A2 - 2 * Y
    dA2_dZ2 = d_logistic(Z2)
    dZ2_dA1 = w_output
    dZ2_dW2 = A1
    dZ2_dB2 = 1
    dA1_dZ1 = d_relu(Z1)
    dZ1_dW1 = X
    dZ1_dB1 = 1
    dC_dW2 = dC_dA2 @ dA2_dZ2 @ dZ2_dW2.T

    dC_dB2 = dC_dA2 @ dA2_dZ2 * dZ2_dB2

    dC_dA1 = dC_dA2 @ dA2_dZ2 @ dZ2_dA1

    dC_dW1 = dC_dA1 @ dA1_dZ1 @ dZ1_dW1.T

    dC_dB1 = dC_dA1 @ dA1_dZ1 * dZ1_dB1

    return dC_dW1, dC_dB1, dC_dW2, dC_dB2

# Ejecutar descenso de gradiente
for i in range(100_000):
    # seleccionar aleatoriamente uno de los datos de entrenamiento
    idx = np.random.choice(n, 1, replace=False)
    X_sample = X_train[idx].transpose()
    Y_sample = Y_train[idx]

    # pasar datos seleccionados aleatoriamente a través de la red neuronal
    Z1, A1, Z2, A2 = forward_prop(X_sample)

    # distribuir error a través de la retropropagación
    # y devolver pendientes para pesos y sesgos
    dW1, dB1, dW2, dB2 = backward_prop(Z1, A1, Z2, A2, X_sample, Y_sample)

    # actualizar pesos y sesgos
    w_hidden -= L * dW1
    b_hidden -= L * dB1
    w_output -= L * dW2
    b_output -= L * dB2

379


In [246]:
def predict_probability(radio_medio, textura_media, perimetro_medio,
                        area_media, suavidad_media, compactacion_media,
                        concavidad_media, puntos_concavos_medios, simetria_media,
                        dimension_fractal_media, radio_se, textura_se, perimetro_se,
                        area_se, suavidad_se, compactacion_se, concavidad_se,
                        concavidad_puntos_se, simetria_se, fractal_dimension_se,
                        radio_peor, textura_peor, perimetro_peor, area_peor,
                        suavidad_peor, compactacion_peor, concavidad_peor,
                        concavidad_punto_peor, simetria_peor, fractal_dimension_peor):
    X = np.array([[radio_medio, textura_media, perimetro_medio,
                   area_media, suavidad_media, compactacion_media,
                   concavidad_media, puntos_concavos_medios, simetria_media,
                   dimension_fractal_media, radio_se, textura_se, perimetro_se,
                   area_se, suavidad_se, compactacion_se, concavidad_se,
                   concavidad_puntos_se, simetria_se, fractal_dimension_se,
                   radio_peor, textura_peor, perimetro_peor, area_peor,
                   suavidad_peor, compactacion_peor, concavidad_peor,
                   concavidad_punto_peor, simetria_peor, fractal_dimension_peor]]).transpose() 
    Z1, A1, Z2, A2 = forward_prop(X)
    return A2



"""""
#opcion1   84358402  0  tiene cancer
radio_medio=20.29
textura_media=14.34
perimetro_medio=135.1
area_media=1297
suavidad_media=0.1003
compactacion_media=0.1328	
concavidad_media=198
puntos_concavos_medios=0.1043
simetria_media=0.1809
dimension_fractal_media=20.30 
radio_se=14.35
textura_se=135.2
perimetro_se=1298
area_se=0.1004
suavidad_se=0.1329
compactacion_se=199
concavidad_se=0.1044
concavidad_puntos_se=0.1810
simetria_se=20.31
fractal_dimension_se=14.36
radio_peor=135.3
textura_peor=1299
perimetro_peor=0.1005
area_peor=0.1330
suavidad_peor=2000
compactacion_peor=0.1045
concavidad_peor=0.1811
concavidad_punto_peor=20.32			
simetria_peor=14.37 
fractal_dimension_peor=135.4
"""""
									
#opcion2 id 8510653 1 no tiene cancer
radio_medio=13.08
textura_media=15.71
perimetro_medio=85.63
area_media=520
suavidad_media=0.1075
compactacion_media=127	
concavidad_media=0.04568
puntos_concavos_medios=0.0311
simetria_media=0.1967
dimension_fractal_media=0.06811
radio_se=0.1852
textura_se=0.7477
perimetro_se=1383
area_se=14.67
suavidad_se=0.004097
compactacion_se=0.01898
concavidad_se=0.01698
concavidad_puntos_se=0.00649
simetria_se=0.01678
fractal_dimension_se=0.002425
radio_peor=14.5
textura_peor=20.49
perimetro_peor=96.09
area_peor=630.5
suavidad_peor=0.1312					

compactacion_peor=0.2776
concavidad_peor=189
concavidad_punto_peor=0.07283		
simetria_peor=0.3184
fractal_dimension_peor=0.08183


# Llama a la función predict_probability con los valores de las características
probabilidad_tener_cancer = predict_probability(radio_medio, textura_media, perimetro_medio,
                        area_media, suavidad_media, compactacion_media,
                        concavidad_media, puntos_concavos_medios, simetria_media,
                        dimension_fractal_media, radio_se, textura_se, perimetro_se,
                        area_se, suavidad_se, compactacion_se, concavidad_se,
                        concavidad_puntos_se, simetria_se, fractal_dimension_se,
                        radio_peor, textura_peor, perimetro_peor, area_peor,
                        suavidad_peor, compactacion_peor, concavidad_peor,
                        concavidad_punto_peor, simetria_peor, fractal_dimension_peor)
if A2 > 0.5:
    # Si la condición es verdadera, ejecutar este bloque de código
    print("tiene cancer.")
else:
    # Si la condición es falsa, ejecutar este bloque de código
    print("no tiene cancer.")



print("Probabilidad de tener cáncer:", probabilidad_tener_cancer)



tiene cancer.
Probabilidad de tener cáncer: [[1.]]
